**The following demonstrates *generalized* Hadamard gates (H)**
For the gate to be generalized, it must be appliable on some target qubit with index q out of n qubits in an arbitrary circuit

We represent the gate generalized as follows:

$$H_k^{n} = I^{\otimes (k-1)} \otimes H \otimes I^{\otimes (n-k)}$$

Where the identity matrix is $$I = \begin{bmatrix} 1 & 0 \\ 0 & 1 \\ \end{bmatrix}$$ and the Hadamard is represented as $$H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$$

In english, this gate is constructed by performing the tensor product of I with itself (k-1) times, then with the Hadamard matrix, and then itself again (n-k) times.

I.e. for a 2-qubit circuit applying H on the second qubit, we would use

$$H_{2}^{2} = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 & 0 & 0 \\ 1 & -1 & 0 & 0 \\ 0 & 0 & 1 & 1 \\ 0 & 0 & 1 & -1 \end{bmatrix}$$

**Be cautious** and note that the qubit index for the second qubit here is 2, but in circuit implementation and typical indexing this qubit would bear *index 1*

We can also define a general Hadamard to apply simultaneously to every qubit in a circuit:

The general hadamard for n qubits is represented as $$H^{\otimes n} = H_0 \otimes H_1 \otimes \cdots \otimes H_n$$

Below is an python routine defining a Hadamard for an n-qubit circuit on some index q

In [313]:
import numpy as np
from matplotlib import pyplot as plt

#hadamard gate for n qubits applied to qubit with index target_q
def hadamard(n, target):
    if target < 0 or target >= n:
        raise ValueError("target_q must be between 0 and n-1.")
    
    H = np.array([[1, 1],
                  [1, -1]], dtype=complex) / np.sqrt(2)
    I = np.eye(2, dtype=complex)

    #Tensor product
    op = 1
    for qubit in range(n):
        if qubit == target:
            op = np.kron(op, H)
        else:
            op = np.kron(op, I)
    
    return op

<h1>Measurement probability</h1>
When a qubit is in superposition, its wavefunction is represented as a superposition of states, each with its own amplitude: $$\ket{\psi} = \alpha_0 \ket{0} + \alpha_1 \ket{1}$$

Since we cannot directly observe the superposition without interacting with the qubit, we must measure it, collapsing it into state $\ket{0}$ or $\ket{1}$

While the initial exact superposition cannot be known, we can use the calculated amplitudes $\alpha_0$ and $\alpha_1$ to find the *probability* of the qubit collapsing into state $\ket{0}$ or $\ket{1}$. Probability $P = \alpha_0^2 + \alpha_1^2 = 1$ represents the sum of all possible state probabilities where $P_{\ket{0}} = \alpha_0^2$ for example.

As probability is a scalar, it is clear to see that we can represent $P$ as an inner product of the wavefunction amplitudes: $$P = \bra{\psi}\ket{\psi} = |\psi|^2$$

<h1>Probability density matrix</h1>

Taking instead the outer product of wavefunction amplitudes, we can also form a *probability density matrix*, which is exceptionally useful for representing the probability states of an entire quantum system, where perhaps some qubits have known states and others are in superposition.

We define this as $\rho$: $$\rho = \ket{\psi}\bra{\psi}$$

Where $Tr(\rho) = 1$

The following routine returns a probability density matrix given a wavefunction

In [ ]:
def prob_density(*amps):
    prob_list = np.array([amps], dtype=complex).reshape(-1,1)
    return prob_list @ prob_list.T

<h1>Generalized CNOT operator</h1>

In an n-qubit circuit, we often ned

In [190]:
def cnot_gate(n, control_q, target_q):
    I = np.eye(2, dtype=complex)
    X = np.array([[0,1],
                  [1,0]], dtype=complex)
    P0 = np.array([[1, 0],
                   [0, 0]], dtype=complex)
    P1 = np.array([[0, 0],
                   [0, 1]], dtype=complex)
    
    #operations for generalized cnot
    ops = []
    for i in range(n):
        if i == control_q:
            ops.append((P0, P1))
        elif i == target_q:
            ops.append((I, X))
        else:
            ops.append((I, I))
    
    term0 = ops[0][0]
    term1 = ops[0][1]
    for i in range(1, n):
        term0 = np.kron(term0, ops[i][0])
        term1 = np.kron(term1, ops[i][1])

    return term0 + term1

In [295]:
#as of now, this routine measures all the qubits and returns measured state
def measure_many(n, amps, target, count):
    #runs measurement routine many times

def measure(n, amps, target):
    #exit routine if target is outside of qubit range
    if target < 0 or target>=n:
        raise ValueError("target needs to be in range [0, n-1]")
        
    #define identity matrix, |0)(0|, |1)(1|
    I = np.array([[1,0],[1,0]], dtype=complex)
    m0 = np.array([[1,0],[0,0]])
    m1 = np.array([[0,0],[0,1]])
    M = 1
    
    #get probability matrix from amplitudes
    p = prob_density(amps)
    print(p)
    
    rand_measure = np.random.choice([0,1], p=[0.5,0.5], size=1)

    for i in range(n):
        if i<target or i>target:
            M = np.kron(M, I)
        else:
            M = np.kron(M, m1 if rand_measure else m0)
    return M
    

def normalize(state):
    norm = np.linalg.norm(state)
    norm = 1 if norm == 0 else norm
    normed = state / norm
    return normed

#demonstrate effect of single hadamard gate and measurement
def run_single_hadamard(qbits, shots):
    measured = np.array([])
    for qbit in qbits:
        appliedH = hadamard(1, target=0) @ qbit
        measured = measure(1, appliedH, 1, shots)
        print("Input: ",qbit.T, "Superposition: ", appliedH.T, "Measurement: ", measured.T[0]) #print horiz for better viewing
    return measured
    
#demonstrate double hadamard and measurement
def run_double_hadamard(qbits, shots):
    measured = np.array([])
    for qbit in qbits:
        appliedH = hadamard(1,target=0) @ qbit
        appliedH2 = hadamard(1,target=0) @ appliedH
        measured = measure(appliedH2, 1, shots)
        print("Input: ",qbit.T, "Superposition: ", appliedH.T, "Measurement: ", measured.T[0])
    return measured

<h2>Demonstration of a hadamard gate on single qubit</h2>
<img src='resources/nate/single_hadamard.png' width="200"/>

The basic hadamard is represented as a 2x2 matrix operator $\frac{1}{\sqrt(2)} \begin{bmatrix}1&1\\1&-1 \end{bmatrix}$

A qubit is provided, initialized at state $\ket{\psi} = \ket{0}$
Hadamard is applied on the qubit, throwing it into superposition $\frac{\ket{0}+\ket{1}}{\sqrt{2}}$

A measurement is then taken, collapsing the qubit to either state $\ket{0} = \begin{bmatrix} 1 \\ 0\end{bmatrix}$ or $\ket{1} = \begin{bmatrix}0\\1\end{bmatrix}$ with probability $|\frac{1}{\sqrt{2}}|^{2} = 0.5$ for both states.

This measurement is represented as matrix multiplication by $\ket{0}\bra{0} = \begin{bmatrix} 1 & 0 \\ 0 & 0\end{bmatrix}$ or $\ket{1}\bra{1} = \begin{bmatrix} 0 & 0 \\ 0 & 1\end{bmatrix}$

In [311]:
#plot results in a bar chart
def plot(measurements):
    #preplot data
    count0 = np.sum(measurements == 0)
    count1 = np.sum(measurements == 1)
    total = count0 + count1
    percent0 = f"{round(count0/total*100, 2)}%"
    percent1 = f"{round(count1/total*100, 2)}%"
    #plots
    plt.bar(['0','1'], [count0, count1], color=['red', 'blue'], label=[percent0, percent1])
    plt.xlabel("Measurement")
    plt.ylabel("Count")
    plt.legend()
    plt.title("Circuit measurements")
    plt.show()

In [312]:
#model qubits as 2x1 array
q0 = np.array([[1],[0]]) #initialized to |0)
q1 = np.array([[0],[1]]) #init to |1)
singleHOutput = run_single_hadamard([q1], 2048)
doubleHOutput = run_double_hadamard([q1], 2048)

#plot outputs
plot(singleHOutput)
plot(doubleHOutput)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

**Entanglement**
The following example serves to show how a few qubits with an entangled state share a wavefunction and thus have correlated outcomes when measuremed

We take three input qubits, superposition one via a Hadamard gate, and entangle them via Controlled-NOT (CNOT) gates.

We use the CNOT gate as a 8x8 matrix with target indices 2 and 3

<img src='resources/nate/entangled.png' width="200"/>

In [94]:
'''
Tensor helper
Usage: kron(one, one, zero) gives |110) basis vector
'''
def kron(*args):
    result = np.array([1], dtype=complex)
    for a in args:
        result = np.kron(result, a)
    return result

In [38]:
'''
Routine to form qubit array for n qubit system
Initializes qubits in 0 states
'''
def init_qubits(n):
    single_state = np.array([[1,0],[0,1]], dtype=complex)
    qubit_matrix = single_state
    for _ in range(n-1):
        qubit_matrix = np.kron(qubit_matrix, single_state)
    return qubit_matrix
init_qubits(2)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [189]:
'''
Routine to get the spanning basis vectors of a C^n hilbert space
for n qubits
WARNING: by nature of the tensor product, the output space will have 2^n bases!
'''
def get_hilbert_basis(num_qubits):
    
    return np.eye(2**(num_qubits), dtype=complex)

In [315]:
'''
Routine to entangle input qubits using hadamard and CNOT
Takes input state |Ψ), applies hadamard to first qubit
Applies CNOT across next qubits in array
'''
def entangle(*qubits):
    num_q = len(qubits) #number of input qubits
    #apply hadamard to first qubit
    qubits[0] = hadamard(num_q) @ qubits[0]
    #successively apply CNOT
    for _ in  range(1, num_q - 1):
        qubits[i] = cnot_gate(num_q, control_q=0, target_q=i) @ qubits[i]
    return qubits


    
**Look up how we can write python code to apply an arbitrary operator on a qubit**

def apply_operator(o, psi, qubit):

    return new_wavefunction

Find a way to represent quantum entanglement and quantum teleportation

In [316]:
m = cnot_gate(3,0,2) @ cnot_gate(3,0,1) @ hadamard(3, 0) @ np.array([[1,0,0,0,0,0,0,0]]).T

In [317]:
get_hilbert_basis(1)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [318]:
one = np.array([[0,1]]).T
zero = np.array([[1,0]]).T
np.kron(one, zero) @ np.kron(one, zero).T

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0]])